In [1]:
import globals
from pathlib import Path
from shutil import rmtree
import os
from os import listdir
from os.path import isfile, join

import sqlite3
from sqlite3 import Error



In [21]:
# Reset the dir that will contain the avg doc vectors
globals.resetDir(globals.avg_doc_vec_dir)

# Get a list of procssed files to iterate over
processed_text_files = globals.getFilesInDir(os.path.join(globals.processed_text_dir, globals.blob_container_path))

# Reset dir that will hold the avg doc term vectors
avg_doc_vec_path = Path(globals.avg_doc_vec_dir)
if avg_doc_vec_path.exists():
    rmtree(globals.avg_doc_vec_dir)
avg_doc_vec_path.mkdir(parents=True)

# Create conn to sqlite to be able to get term vectors and avg bm25 values
conn = sqlite3.connect(os.path.join(globals.sqlite_term_vec_dir,globals.sqlite_term_vec_file))


In [ ]:
print ('Processing files...')
with open(os.path.join(globals.avg_doc_vec_dir, globals.avg_doc_vec_file), 'w', encoding='utf-8') as f:
    for file in processed_text_files:
        # Open the file in read mode 
        # print ('Processing: ', file)
        text = open(os.path.join(globals.processed_text_dir, globals.blob_container_path, file), "r", encoding='utf-8') 
        
        # Remove the leading spaces and newline character 
        line = text.read().strip() 
        
        # Split the line into words 
        words = line.split(" ") 

        # Iterate over each word in line 
        filter_sql = ''
        for word in words: 
            filter_sql += "'" + word.replace("'", "''") + "' "

        sql = 'select term, avgbm25, vector from vectors where term in ' + '(' + filter_sql.replace("' '", "', '", ) + ')'

        cur = conn.cursor()
        cur.execute(sql)

        rows = cur.fetchall()

        vectors = []

        sum_vectors = dict()
        for sv in range(0,300):
            sum_vectors[sv] = 0


        # This is where I will average the vectors, however I will give more weighting to vectors 
        # with higher average bm25

        row_counter = 0
        for row in rows:
            avgbm25 = int(round(row[1]))
            if avgbm25 < 1:
                avgbm25 = 1

            vec_fields = str(row[2]).split('|')
            field_counter = 0
            if vec_fields[0] != 'None':
                for v in vec_fields[0:300]:
                    sum_vectors[field_counter] = sum_vectors[field_counter] + (float(v) * avgbm25)
                    field_counter += 1

            row_counter += 1 * avgbm25

        avg_vector = ''
        for v in sum_vectors.keys():
            avg_vector += str(sum_vectors[v] / row_counter) + '|'

        f.write (file.replace('.txt', '') + '\t' + avg_vector + '\r\n')

Processing:  71469.txt
Processing:  6910.txt
Processing:  17258.txt
Processing:  66382.txt
Processing:  181979.txt
Processing:  34148.txt
Processing:  63352.txt
Processing:  10617.txt
Processing:  125114.txt
Processing:  184203.txt
Processing:  205064.txt
Processing:  186265.txt
Processing:  124122.txt
Processing:  70457.txt
Processing:  57001.txt
Processing:  107029.txt
Processing:  117482.txt
Processing:  98305.txt
Processing:  71068.txt
Processing:  137027.txt
Processing:  156862.txt
Processing:  106730.txt
Processing:  221130.txt
Processing:  211458.txt
Processing:  147452.txt
Processing:  185032.txt
Processing:  77008.txt
Processing:  44304.txt
Processing:  54888.txt
Processing:  223317.txt
Processing:  147135.txt
Processing:  138244.txt
Processing:  156771.txt
Processing:  91938.txt
Processing:  219144.txt
Processing:  152976.txt
Processing:  3732.txt
Processing:  42649.txt
Processing:  212473.txt
Processing:  114370.txt
Processing:  14990.txt
Processing:  66423.txt
Processing:  